# PoC #10: AI-Powered Resume Screening 📄

## Business Use Case
Automate initial resume screening to reduce hiring time by 70% while ensuring fair, consistent candidate evaluation.

## Production Applications
- Applicant tracking systems (ATS)
- Recruitment automation
- Talent pipeline management
- Skills assessment

---

In [ ]:
!pip install -qU "ibm-watsonx-ai>=1.1.22" pandas
print("✅ Ready")

In [ ]:
import getpass
import pandas as pd
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

model = Model(
    model_id="ibm/granite-3-8b-instruct",
    credentials=Credentials(url="https://us-south.ml.cloud.ibm.com", api_key=getpass.getpass("API Key: ")),
    project_id=getpass.getpass("Project ID: "),
    params={GenParams.MAX_NEW_TOKENS: 200, GenParams.TEMPERATURE: 0.2}
)
print("✅ Resume screener ready")

In [ ]:
def screen_resume(resume_text: str, job_requirements: str) -> dict:
    prompt = f"""Evaluate this resume against job requirements. Respond in this format:

Match Score: [0-100]
Strengths: [2-3 bullet points]
Gaps: [1-2 bullet points]
Recommendation: [STRONG_MATCH / POTENTIAL_MATCH / NO_MATCH]

Job Requirements:
{job_requirements}

Resume:
{resume_text}

Evaluation:"""
    
    evaluation = model.generate_text(prompt=prompt)
    
    # Extract match score if present
    try:
        score_line = [line for line in evaluation.split('\n') if 'Match Score' in line][0]
        score = int(''.join(filter(str.isdigit, score_line)))
    except:
        score = 0
    
    return {
        "evaluation": evaluation,
        "score": score
    }

In [ ]:
# Job posting
job_requirements = """Senior Python Developer

Required Skills:
- 5+ years Python development
- Experience with Django/Flask
- SQL databases (PostgreSQL/MySQL)
- RESTful API design
- Git version control

Nice to Have:
- Cloud platforms (AWS/Azure)
- Docker/Kubernetes
- Machine learning experience
"""

# Mock resumes
resumes = [
    {
        "name": "Alice Johnson",
        "text": """Senior Software Engineer with 7 years experience in Python development. 
Expert in Django and Flask frameworks. Extensive PostgreSQL experience.
Built and maintained RESTful APIs serving 1M+ requests/day.
Proficient with Git, Docker, and AWS deployment."""
    },
    {
        "name": "Bob Smith",
        "text": """Junior Developer with 2 years experience. 
Familiar with Python basics and MySQL.
Completed online courses in web development.
Looking to grow skills in backend development."""
    },
    {
        "name": "Carol Williams",
        "text": """Full-Stack Developer with 6 years Python experience.
Built production systems using Flask and FastAPI.
Experience with PostgreSQL, MongoDB.
Deployed applications on AWS using Docker/Kubernetes.
Contributed to open-source ML projects."""
    }
]

print(f"🔍 Screening {len(resumes)} resumes...\n" + "="*80)

In [ ]:
results = []

for resume in resumes:
    print(f"\n\n👤 Candidate: {resume['name']}")
    print("="*80)
    
    result = screen_resume(resume['text'], job_requirements)
    results.append({
        "name": resume['name'],
        "score": result['score'],
        "evaluation": result['evaluation']
    })
    
    print(result['evaluation'])
    print("\n" + "-"*80)

In [ ]:
# Ranking
df = pd.DataFrame(results).sort_values('score', ascending=False)

print("\n\n📊 CANDIDATE RANKING")
print("="*80)
for idx, row in df.iterrows():
    print(f"{row['name']:20} | Score: {row['score']:3}/100")
print("="*80)

print(f"\n✅ Top candidate: {df.iloc[0]['name']} (Score: {df.iloc[0]['score']})")

In [ ]:
# Screen your resume
print("\nPaste your resume text (press Enter twice when done):\n")
lines = []
while True:
    line = input()
    if line == "": break
    lines.append(line)

your_resume = "\n".join(lines)

if your_resume.strip():
    result = screen_resume(your_resume, job_requirements)
    print(f"\n{'='*80}\n📋 YOUR RESUME EVALUATION\n{'='*80}\n")
    print(result['evaluation'])
    print(f"\n{'='*80}")

---

## Production Benefits
- ⏱️ 70% faster initial screening
- ⚖️ Consistent, bias-free evaluation
- 📊 Quantifiable candidate ranking
- 🎯 Focus on top candidates

### Ethical AI Considerations
1. **Fairness**: Regular bias audits
2. **Transparency**: Explainable scoring
3. **Human oversight**: Final decisions by recruiters
4. **Privacy**: Secure data handling
5. **Compliance**: GDPR/EEOC compliance

### Production Integration
```python
# Example integrations:
# - Greenhouse API
# - Lever ATS
# - Workday Recruiting
# - Custom ATS webhooks
```

---